In [ ]:
! pip install -U sentence-transformers
! pip install pinecone-client

In [ ]:
import csv
from itertools import islice

#Load data
batch_input = []
with open('/content/drive/MyDrive/data/INLPT_data/articles.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        batch_input.append((row["PMID"],row["TI"],row["AB"],row["PB"],row["FAU"],row["FED"],row["DP"],row["OTO"],row["OT"],row["OWN"],row["DCOM"],row["LR"],row["JT"],row["MH"],row["ISBN"]))

In [ ]:
from sentence_transformers import SentenceTransformer

#Embed Abstract text which is AB column
model = SentenceTransformer('pritamdeka/S-PubMedBert-MS-MARCO')
embeddings = [model.encode(chunk[2]) for chunk in batch_input]


In [ ]:
import uuid
vectors = []

createvec = [{vectors.append((
    str(uuid.uuid1()),
    embeddings[i].tolist(),
    {"PMID": data[0], "TI": data[1],"AB": data[2], "PB": data[3], "FAU": data[4],"FED": data[5],"DP": data[6],"OTO": data[7],"OT": data[8],"OWN": data[9],"DCOM": data[10],"LR": data[11],"JT": data[12],"MH": data[13],"ISBN": data[14]}
))} for i, data in enumerate(batch_input)]

In [ ]:
import pinecone

#Init pinecone index
pinecone.init(api_key="4d2c2cd0-cf55-43c5-afb1-11001dc68709", environment="gcp-starter")
index = pinecone.Index("inlp-ws2324-med")

Problem: storing abstract as metadata doesn't work: metadata size limit is 40kB, ours partially exceed by up to 60 kB \n

Solutions:
- Upload embeddings + id into Pinecone, Upload text, metadata etc. into 2. Database (f.e. MongoDB). Once data is retrieved from pinecone via similarity search query text via id from 2. DB
- Split up abstracts 
- use different cloud based vector db

In [ ]:
import random
import itertools

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

vector_dim = 768

# Upsert data with 100 vectors per upsert request
for ids_vectors_chunk in chunks(vectors, batch_size=100):
    index.upsert(vectors=ids_vectors_chunk)